## Import libraries

In [74]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [75]:
train_data = np.load('dataset/Shoes - All - Train.npz')
validation_data = np.load('dataset/Shoes - All - Validation.npz')
test_data = np.load('dataset/Shoes - All - Test.npz')

In [76]:
train_data.files

['images', 'labels', 'genders']

In [77]:
images_train, genders_train = train_data['images'], train_data['genders']
images_val, genders_val = validation_data['images'], validation_data['genders']
images_test, genders_test = test_data['images'], test_data['genders']

In [78]:
images_train.shape

(4959, 120, 90, 3)

In [79]:
genders_train.shape

(4959,)

In [80]:
# Data augmentation
# Flip the image by flip the axis 2
# we have 4 axis now in images (no. of pic, y, x, channel)
# so, if we want to flip horizontally, we flip axis 2

flipped_images_train = np.flip(images_train, axis = 2)
flipped_images_val = np.flip(images_val, axis = 2)
flipped_images_test = np.flip(images_test, axis = 2)

In [81]:
# Don't forget to concatenate labels with itself, cuz we increase pic in images, so labels have to be increased too.
images_train = np.concatenate((images_train, flipped_images_train))
genders_train = np.concatenate((genders_train, genders_train))

images_val = np.concatenate((images_val, flipped_images_val))
genders_val = np.concatenate((genders_val, genders_val))

images_test = np.concatenate((images_test, flipped_images_test))
genders_test = np.concatenate((genders_test, genders_test))

In [82]:
images_val.shape

(1238, 120, 90, 3)

In [83]:
genders_val.shape

(1238,)

In [84]:
images_test.shape

(1238, 120, 90, 3)

In [85]:
genders_test.shape

(1238,)

In [86]:
np.unique(genders_train)

array([0, 1])

In [87]:
# Standardize the data
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Model

### Define hyperparameters

In [88]:
EPOCHS = 15
BATCH_SIZE = 64

In [89]:
HP_KERNEL_SIZE = hp.HParam('kernel size', hp.Discrete([3,5]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64,128]))
HP_LAMBDA_REG = hp.HParam('lambda reg', hp.Discrete([1e-5,1e-4,1e-3]))

In [90]:
METRIC = 'accuracy'

with tf.summary.create_file_writer(r'logs/Model 1 (Shoes_Gender_L2)/hparam_tuning').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE, HP_KERNEL_NUM, HP_LAMBDA_REG],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [91]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', input_shape = (120,90,3), kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.Dense(2, activation = 'softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(name = 'sparse_categorical_crossentropy')
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy','sparse_categorical_crossentropy'])
    
    log_dir = "Logs\\Model 1 (Shoes_Gender_L2)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(genders_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Male','Female'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_sparse_categorical_crossentropy', patience = 2, restore_best_weights = True)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    
    model.fit(images_train,
              genders_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, genders_val),
              verbose = 1)
    
    _, accuracy, _ = model.evaluate(images_val, genders_val)
    
    model.save(r'saved_models\Model 1 (Shoes_Gender_L2)\Run-{}'.format(session_num))
    
    return accuracy

In [92]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [93]:
session_num = 1
for kernel_size in HP_KERNEL_SIZE.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for lambda_reg in HP_LAMBDA_REG.domain.values:
            hparams = {
                HP_KERNEL_SIZE : kernel_size,
                HP_KERNEL_NUM : kernel_num,
                HP_LAMBDA_REG : lambda_reg
            }
            run_name = f'run-{session_num}'
            print('--- Starting Trial:',run_name)
            print({h.name: hparams[h] for h in hparams})
            run("Logs/Model 1 (Shoes_Gender_L2)/hparam_tuning/" + run_name, hparams, session_num)

            session_num += 1

--- Starting Trial: run-1
{'kernel size': 3, 'kernel num': 32, 'lambda reg': 1e-05}
Epoch 1/15
155/155 [==============================] - 51s 329ms/step - loss: 0.5942 - accuracy: 0.7309 - sparse_categorical_crossentropy: 0.5892 - val_loss: 0.5210 - val_accuracy: 0.7488 - val_sparse_categorical_crossentropy: 0.5159
Epoch 2/15
155/155 [==============================] - 51s 326ms/step - loss: 0.4131 - accuracy: 0.8107 - sparse_categorical_crossentropy: 0.4077 - val_loss: 0.4090 - val_accuracy: 0.8118 - val_sparse_categorical_crossentropy: 0.4032
Epoch 3/15
155/155 [==============================] - 54s 349ms/step - loss: 0.3662 - accuracy: 0.8366 - sparse_categorical_crossentropy: 0.3599 - val_loss: 0.4054 - val_accuracy: 0.8166 - val_sparse_categorical_crossentropy: 0.3987
Epoch 4/15
155/155 [==============================] - 55s 352ms/step - loss: 0.3311 - accuracy: 0.8532 - sparse_categorical_crossentropy: 0.3240 - val_loss: 0.4173 - val_accuracy: 0.8223 - val_sparse_categorical_cross

Epoch 5/15
155/155 [==============================] - 256s 2s/step - loss: 0.4275 - accuracy: 0.8329 - sparse_categorical_crossentropy: 0.3672 - val_loss: 0.4683 - val_accuracy: 0.8142 - val_sparse_categorical_crossentropy: 0.4089
Epoch 6/15
155/155 [==============================] - 256s 2s/step - loss: 0.4172 - accuracy: 0.8366 - sparse_categorical_crossentropy: 0.3561 - val_loss: 0.4702 - val_accuracy: 0.8183 - val_sparse_categorical_crossentropy: 0.4075
Epoch 7/15
155/155 [==============================] - 256s 2s/step - loss: 0.4063 - accuracy: 0.8474 - sparse_categorical_crossentropy: 0.3431 - val_loss: 0.4555 - val_accuracy: 0.8247 - val_sparse_categorical_crossentropy: 0.3914
Epoch 8/15
155/155 [==============================] - 255s 2s/step - loss: 0.3849 - accuracy: 0.8537 - sparse_categorical_crossentropy: 0.3194 - val_loss: 0.4530 - val_accuracy: 0.8304 - val_sparse_categorical_crossentropy: 0.3876
Epoch 9/15
155/155 [==============================] - 252s 2s/step - loss: 0

Epoch 1/15
155/155 [==============================] - 170s 1s/step - loss: 0.7242 - accuracy: 0.7222 - sparse_categorical_crossentropy: 0.6860 - val_loss: 0.5080 - val_accuracy: 0.7795 - val_sparse_categorical_crossentropy: 0.4763
Epoch 2/15
155/155 [==============================] - 170s 1s/step - loss: 0.4914 - accuracy: 0.7902 - sparse_categorical_crossentropy: 0.4624 - val_loss: 0.4802 - val_accuracy: 0.7876 - val_sparse_categorical_crossentropy: 0.4534
Epoch 3/15
155/155 [==============================] - 171s 1s/step - loss: 0.4448 - accuracy: 0.8083 - sparse_categorical_crossentropy: 0.4197 - val_loss: 0.4540 - val_accuracy: 0.8029 - val_sparse_categorical_crossentropy: 0.4303
Epoch 4/15
155/155 [==============================] - 170s 1s/step - loss: 0.4090 - accuracy: 0.8243 - sparse_categorical_crossentropy: 0.3859 - val_loss: 0.4444 - val_accuracy: 0.7956 - val_sparse_categorical_crossentropy: 0.4220
Epoch 5/15
155/155 [==============================] - 171s 1s/step - loss: 0